# Aula 1 - Buscando a similaridade

In [ ]:
url = 'https://raw.githubusercontent.com/Mirlaa/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [ ]:
import pandas as pd

dados = pd.read_csv(url)
dados

,nota_review,review
0,5,Recebi bem antes do prazo estipulado.
1,5,Parabéns lojas Zoop adorei comprar pela Intern...
2,4,aparelho eficiente. no site a marca do aparelh...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,5,"Vendedor confiável, produto ok e entrega antes..."
...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...
39458,3,"O produto não foi enviado com NF, não existe v..."
39459,5,"Excelente mochila, entrega super rápida. Super..."
39460,1,Solicitei a compra de uma capa de retrovisor c...


In [ ]:
dados['review'][1]

'Parabéns lojas Zoop adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa'

In [ ]:
dados['review'][8]

'A compra foi realizada facilmente.\r\nA entrega foi efetuada muito antes do prazo dado.\r\nO produto já começou a ser usado e até o presente,\r\nsem problemas.'

In [ ]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [ ]:
dados['review'][18]

'otimo vendedor chegou ate antes do prazo , adorei o produto'

## Similaridade entre as palavras

In [ ]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language = 'portuguese')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences = dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

In [ ]:
modelo.wv.most_similar('entrega')

[('Entrega', 0.7526029944419861),
 ('entregue', 0.7109662294387817),
 ('compra', 0.6985824108123779),
 ('chegou', 0.6755236983299255),
 ('entregou', 0.6542965769767761),
 ('envio', 0.6249788403511047),
 ('cadastrado', 0.6161651611328125),
 ('Chegou', 0.6127404570579529),
 ('preparar', 0.6111896634101868),
 ('mercadoria', 0.6036772727966309)]

In [ ]:
modelo.wv.most_similar('vendedor')

[('atendimento', 0.8607712388038635),
 ('serviço', 0.8035672307014465),
 ('fornecedor', 0.7637899518013),
 ('veiculo', 0.7295382022857666),
 ('produto', 0.729019284248352),
 ('processo', 0.710643470287323),
 ('parceiro', 0.702766478061676),
 ('cumpriu', 0.6967913508415222),
 ('trabalho', 0.6896953582763672),
 ('cumpriram', 0.6866145133972168)]

In [ ]:
modelo.wv.most_similar('produto')

[('relógio', 0.7635505199432373),
 ('pedido', 0.7503548860549927),
 ('vendedor', 0.729019284248352),
 ('endereço', 0.7101667523384094),
 ('Produto', 0.6988608837127686),
 ('mesmo', 0.6944952607154846),
 ('tempo', 0.678637683391571),
 ('Mas', 0.673784077167511),
 ('porém', 0.6504761576652527),
 ('mercadoria', 0.6491450667381287)]

## Similaridade entre sentenças

In [ ]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [ ]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]

In [ ]:
dados_tag

[TaggedDocument(words=['Recebi', 'bem', 'antes', 'do', 'prazo', 'estipulado', '.'], tags=['0']),
 TaggedDocument(words=['Parabéns', 'lojas', 'Zoop', 'adorei', 'comprar', 'pela', 'Internet', 'seguro', 'e', 'prático', 'Parabéns', 'a', 'todos', 'feliz', 'Páscoa'], tags=['1']),
 TaggedDocument(words=['aparelho', 'eficiente', '.', 'no', 'site', 'a', 'marca', 'do', 'aparelho', 'esta', 'impresso', 'como', '3desinfector', 'e', 'ao', 'chegar', 'esta', 'com', 'outro', 'nome', '...', 'atualizar', 'com', 'a', 'marca', 'correta', 'uma', 'vez', 'que', 'é', 'o', 'mesmo', 'aparelho'], tags=['2']),
 TaggedDocument(words=['Mas', 'um', 'pouco', ',', 'travando', '...', 'pelo', 'valor', 'ta', 'Boa', '.'], tags=['3']),
 TaggedDocument(words=['Vendedor', 'confiável', ',', 'produto', 'ok', 'e', 'entrega', 'antes', 'do', 'prazo', '.'], tags=['4']),
 TaggedDocument(words=['GOSTARIA', 'DE', 'SABER', 'O', 'QUE', 'HOUVE', ',', 'SEMPRE', 'RECEBI', 'E', 'ESSA', 'COMPRA', 'AGORA', 'ME', 'DECPCIONOU'], tags=['5']),
 T

In [ ]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)

In [ ]:
vetor_inferido = modelo.infer_vector(['entrega'])
vetor_inferido

array([-1.64769620e-01, -4.19039913e-02,  4.54760119e-02, -3.79462056e-02,
        5.01080491e-02,  9.11147520e-02, -6.34148195e-02, -1.34524927e-02,
       -1.44093094e-04, -8.93729702e-02, -8.32309127e-02,  6.39296416e-03,
       -1.13408774e-01,  2.26794947e-02, -5.24243526e-02, -8.90790075e-02,
        4.26806416e-03,  5.67897819e-02, -7.33350264e-03, -4.51776519e-04,
        2.68019903e-02, -4.48587239e-02,  6.47819713e-02,  2.54743416e-02,
        4.34280699e-03, -5.58641972e-03, -1.45101354e-01, -5.51957004e-02,
       -2.37516519e-02, -4.98959906e-02,  8.09897929e-02,  3.47317606e-02,
        2.39257645e-02,  7.19653517e-02,  4.09114398e-02, -8.42812844e-03,
       -8.25305060e-02,  1.40931830e-01, -1.23146042e-01, -1.24326143e-02,
        8.56279396e-04,  1.19140306e-02,  2.38020960e-02, -6.98463097e-02,
       -6.39492720e-02, -3.93858925e-02,  2.58012526e-02, -5.63793350e-03,
       -8.53652507e-02, -2.06154697e-02,  3.98675874e-02,  1.44594358e-02,
        1.33702401e-02, -

In [ ]:
frases_similares = modelo.dv.most_similar([vetor_inferido],topn=5)
frases_similares

[('24666', 0.8576596975326538),
 ('9304', 0.8565875887870789),
 ('33496', 0.8525092601776123),
 ('7314', 0.845014750957489),
 ('15598', 0.8377625942230225)]

In [ ]:
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['review']} - Similaridade: {similaridade:.4f}")

Review: entrega antes do prazo - Similaridade: 0.8577
Review: entrega no prazo. - Similaridade: 0.8566
Review: Ainda estou esperando a entrega!  - Similaridade: 0.8525
Review: entrega antes do prazo. - Similaridade: 0.8450
Review: Rever a política de entrega  - Similaridade: 0.8378


# Aula 2 - Ajustando os textos de review

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
from nltk.corpus import stopwords

def tratamento_inicial (texto):
  texto = re.sub(r'\W',' ',texto.lower())

  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')

  return ' '.join([w for w in tokens if w not in stop_words])

In [ ]:
dados['tratamento_1'] = dados['review'].apply(tratamento_inicial)

In [ ]:
dados['tratamento_1']

,tratamento_1
0,recebi bem antes prazo estipulado
1,parabéns lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiável produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condições...
39458,produto não enviado nf não existe venda nf cer...
39459,excelente mochila entrega super rápida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [ ]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [ ]:
dados['tratamento_1'][11]

'sempre compro internet entrega ocorre antes prazo combinado acredito prazo máximo prazo máximo esgotou ainda não recebi produto'

## Uniformizando o texto

In [ ]:
!pip install unidecode

In [ ]:
import unidecode

sem_acentos = [unidecode.unidecode(texto) for texto in dados['tratamento_1']]

In [ ]:
dados['tratamento_2'] = sem_acentos
dados['tratamento_2']

,tratamento_2
0,recebi bem antes prazo estipulado
1,parabens lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiavel produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condicoes...
39458,produto nao enviado nf nao existe venda nf cer...
39459,excelente mochila entrega super rapida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [ ]:
dados['review_token'] = dados['tratamento_2'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['tratamento_2']} - Similaridade: {similaridade:.4f}")

Review: entrega antes prazo - Similaridade: 0.8671
Review: entrega antes prazo - Similaridade: 0.8662
Review: prazo entrega gigante assim conseguiram atrasar entrega - Similaridade: 0.8636
Review: entrega prazo produto conforme solicitado - Similaridade: 0.8619
Review: entrega prazo produto dentro esperado - Similaridade: 0.8570


## Correções nos dados

In [ ]:
df = dados.drop_duplicates(subset= ['tratamento_2'])
df

,nota_review,review,review_token,tratamento_1,tratamento_2
0,5,Recebi bem antes do prazo estipulado.,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[parabens, lojas, zoop, adorei, comprar, inter...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, site, marca, aparelho, i...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[pouco, travando, valor, ta, boa]",pouco travando valor ta boa,pouco travando valor ta boa
4,5,"Vendedor confiável, produto ok e entrega antes...","[vendedor, confiavel, produto, ok, entrega, an...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo
...,...,...,...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...
39458,3,"O produto não foi enviado com NF, não existe v...","[produto, nao, enviado, nf, nao, existe, venda...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...
39459,5,"Excelente mochila, entrega super rápida. Super...","[excelente, mochila, entrega, super, rapida, s...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...
39460,1,Solicitei a compra de uma capa de retrovisor c...,"[solicitei, compra, capa, retrovisor, celta, p...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...


In [ ]:
df['review'][120]

'A'

In [ ]:
df['tratamento_2'][120]

''

In [ ]:
df = df[df['tratamento_2'] != '']

In [ ]:
df['tratamento_2'][3319]

'simplismeente ameiii'

In [ ]:
df['tratamento_2'][8354]

'recebi rapido antes prazo otimo qualidade adoreii'

In [ ]:
df['tratamento_2'][12843]

'produto chegou direitinho antes muuuito antes prazo superou expectativas'

In [ ]:
def normalizar_repeticoes(texto):
  return re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto)

df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)

<ipython-input-37-696f58db5688>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)


In [ ]:
df['tratamento_3'][3319]

'simplismente amei'

In [ ]:
df['tratamento_3'][8354]

'recebi rapido antes prazo otimo qualidade adorei'

In [ ]:
df.reset_index(drop=True, inplace=True)

## Lematização

In [ ]:
!pip install stanza
!python -m stanza.download pt

/usr/bin/python3: No module named stanza.download


In [ ]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt',
                      processors='tokenize,lemma',
                      use_gpu=False,
                      batch_size= 64,
                      n_process = 4)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: pt (Portuguese) ...
INFO:stanza:File exists: /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package         |
-------------------------------
| tokenize  | bosque          |
| mwt       | bosque          |
| lemma     | bosque_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this

In [ ]:
def lematizar_texto(textos):
  texto_lematizados = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizados.append(frase_lematizada)

  return texto_lematizados

In [ ]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']
lematizar_texto(textos)

['gostar muito experiencia comprar',
 'meu filha gostar produto',
 'compra ser facil compra rapido']

In [ ]:
df['tratamento_4'] = lematizar_texto(df['tratamento_3'])

<ipython-input-45-5e1b48a16cfb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_4'] = lematizar_texto(df['tratamento_3'])


In [ ]:
df['tratamento_4']

,tratamento_4
0,recebi bem antes prazo estipulado
1,parabem loja zop adorei comprar internet segur...
2,aparelho eficiente site marca aparelho imprimi...
3,pouco travar valor to bom
4,vendedor confiavel produto ok entregar antes p...
...,...
31469,entregar dentro prazo produto chegar condico p...
31470,produto nao enviado em f nao existir venda em ...
31471,excelente mochila entregar super rapido super ...
31472,solicitei compra capa retrovisor celta prisma ...


# Aula 3 - Otimizando o Doc2vec

In [ ]:
df_lem = df.drop_duplicates(subset=['tratamento_4'])
df_lem

,nota_review,review,review_token,tratamento_1,tratamento_2,tratamento_3,tratamento_4
0,5,Recebi bem antes do prazo estipulado.,"[recebi, bem, antes, prazo, estipulado]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,recebi bem antes prazo estipulado
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[parabens, lojas, zoop, adorei, comprar, inter...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...,parabens lojas zop adorei comprar internet seg...,parabem loja zop adorei comprar internet segur...
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, site, marca, aparelho, i...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho imprimi...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[pouco, travando, valor, ta, boa]",pouco travando valor ta boa,pouco travando valor ta boa,pouco travando valor ta boa,pouco travar valor to bom
4,5,"Vendedor confiável, produto ok e entrega antes...","[vendedor, confiavel, produto, ok, entrega, an...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,vendedor confiavel produto ok entregar antes p...
...,...,...,...,...,...,...,...
31469,5,Entregou dentro do prazo. O produto chegou em ...,"[entregou, dentro, prazo, produto, chegou, con...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...,entregou dentro prazo produto chegou condicoes...,entregar dentro prazo produto chegar condico p...
31470,3,"O produto não foi enviado com NF, não existe v...","[produto, nao, enviado, nf, nao, existe, venda...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,produto nao enviado em f nao existir venda em ...
31471,5,"Excelente mochila, entrega super rápida. Super...","[excelente, mochila, entrega, super, rapida, s...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...,excelente mochila entrega super rapida super r...,excelente mochila entregar super rapido super ...
31472,1,Solicitei a compra de uma capa de retrovisor c...,"[solicitei, compra, capa, retrovisor, celta, p...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...


In [ ]:
df_lem.reset_index(drop=True, inplace=True)

In [ ]:
df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in df_lem.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers= 1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)
for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_4']} - Similaridade: {similaridade:.4f}")

<ipython-input-49-24e313764618>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)


Review: muito desencontro entrega produto - Similaridade: 0.8857
Review: entrega demorando - Similaridade: 0.8686
Review: correio sairam greve - Similaridade: 0.8595
Review: adoro compro sempre - Similaridade: 0.8577
Review: entregar atrasar - Similaridade: 0.8521


## Treinamento do Doc2vec

In [ ]:
modelo = Doc2Vec(vector_size = 300, min_count = 2,
                 window=5, workers = 1, seed=45, epochs = 15)
modelo.build_vocab(dados_tag)

modelo.train(dados_tag, total_examples=modelo.corpus_count,
             epochs= modelo.epochs)

## Verificando a similardade - Doc2Vec

In [ ]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])],topn=5)

for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_1']} - Similaridade: {similaridade:.4f}")

Review: ultimamente atrasando entregas - Similaridade: 0.9480
Review: compro sempre - Similaridade: 0.9349
Review: gostaria entregas rápidas - Similaridade: 0.9254
Review: embalagem veio violada - Similaridade: 0.9250
Review: muitos desencontros entregas produtos - Similaridade: 0.9235


In [ ]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['vendedor'])],topn=5)

for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_1']} - Similaridade: {similaridade:.4f}")

Review: vendedor bom - Similaridade: 0.9413
Review: nota 10 vendedor - Similaridade: 0.9359
Review: vendedor pontualíssimo recomendo - Similaridade: 0.9326
Review: produto entregue dentro prazo estipulado vendedor - Similaridade: 0.9325
Review: compraria novamente vendedor - Similaridade: 0.9280


In [ ]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['reputacao loja'])],topn=5)

for idx, similaridade in frases_similares:
    print(f"Review: {df_lem.iloc[int(idx)]['tratamento_1']} - Similaridade: {similaridade:.4f}")

Review: atendimento perfeito dando garantia cliente - Similaridade: 0.2342
Review: ótimo produto veio perfeitamente embalado atende todas necessidades precisávamos loja atenciosa - Similaridade: 0.2280
Review: tudo ok rápido entrega bem embalado recebimento todos grupo baratheon ob - Similaridade: 0.2199
Review: adquiri duas memórias pc antigo recebi defeito entrei contato através duas mensagens deram trabalho responder decepcionou - Similaridade: 0.2194
Review: bem embalado aplicação não tão difícil realmente feito 2 pessoas veio coloração certa adesivo gruda rápido momento não descolou nenhuma ponta recomendo - Similaridade: 0.2133


# Aula 4 - Considerando o significado das frases

In [ ]:
modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

In [ ]:
import tensorflow_hub as hub

modelo = hub.load(modulo_url)

In [ ]:
modelo

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7b6925a2fe20>

In [ ]:
reviews = df['tratamento_3'].tolist()
reviews_emb = modelo(reviews)
reviews_emb

<tf.Tensor: shape=(31474, 512), dtype=float32, numpy=
array([[ 0.04775169,  0.01094663, -0.0541508 , ...,  0.09078011,
        -0.02599917, -0.09261213],
       [-0.00746649, -0.06605157, -0.04505714, ...,  0.05561579,
        -0.02624756, -0.07801282],
       [-0.00820429,  0.04282852, -0.02907449, ...,  0.07496072,
        -0.06071885, -0.07618523],
       ...,
       [ 0.02074313, -0.0543616 , -0.06415588, ...,  0.0719423 ,
         0.0105929 , -0.07603392],
       [-0.04131037, -0.03054746,  0.01322938, ...,  0.06785525,
        -0.0252889 , -0.07106064],
       [ 0.05717909, -0.05808825, -0.02038886, ...,  0.07339787,
        -0.06409648, -0.08014238]], dtype=float32)>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(reviews_emb)

array([[1.0000002 , 0.61586297, 0.5175547 , ..., 0.5427756 , 0.57541025,
        0.6691027 ],
       [0.61586297, 0.99999994, 0.5666107 , ..., 0.63920605, 0.6011834 ,
        0.6711744 ],
       [0.5175547 , 0.5666107 , 0.9999999 , ..., 0.58997375, 0.6564975 ,
        0.5922985 ],
       ...,
       [0.5427756 , 0.63920605, 0.58997375, ..., 1.        , 0.6378722 ,
        0.61628115],
       [0.57541025, 0.6011834 , 0.6564975 , ..., 0.6378722 , 1.0000001 ,
        0.6389235 ],
       [0.6691027 , 0.6711744 , 0.5922985 , ..., 0.61628115, 0.6389235 ,
        1.        ]], dtype=float32)

## Avaliando as sentenças similares

In [ ]:
import numpy as np
def sentencas_similares(tema, reviews, reviews_emb, top_n=5):
  tema_emb = modelo([tema])
  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()

  indices_similares = np.argsort(-similaridades)
  for idx in indices_similares[:top_n]:
      print(f'Review: {reviews[idx]} - Similaridade: {similaridades[idx]:.4f}')

In [ ]:
sentencas_similares('entrega', reviews, reviews_emb)

Review: eficiente entrega - Similaridade: 0.6829
Review: entrega eficiente - Similaridade: 0.6756
Review: agil entrega - Similaridade: 0.6563
Review: entrega prazo - Similaridade: 0.6473
Review: entrega agil - Similaridade: 0.6471


In [ ]:
sentencas_similares('reputação loja', reviews, reviews_emb, top_n=10)

Review: otima loja - Similaridade: 0.8578
Review: recomento loja - Similaridade: 0.8339
Review: pessima loja - Similaridade: 0.8335
Review: adorei loja - Similaridade: 0.8170
Review: rapidez eficiencia loja - Similaridade: 0.8018
Review: loja confianca - Similaridade: 0.7909
Review: produto loja confianca - Similaridade: 0.7888
Review: loja ruim - Similaridade: 0.7868
Review: otimas experiencias loja - Similaridade: 0.7839
Review: satisfeita produto loja - Similaridade: 0.7816


# Aula 5 - Transformando a similaridade em aplicação

In [ ]:
# !pip install gradio

In [ ]:
import gradio as gr

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
modelo = hub.load(modulo_url)

def tratar_texto(texto):
  texto = re.sub(r'\W', ' ', texto.lower())

  word_tokenize_pt = partial(word_tokenize, language='portuguese')
  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')
  tokens = [w for w in tokens if w not in stop_words]

  texto_sem_acentos = unidecode.unidecode(' '.join(tokens))

  texto_normalizado = re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto_sem_acentos)
  return texto_normalizado

def process_csv(arquivo, tema):
  df = pd.read_csv(arquivo.name)

  df['review_tratada'] = df['review'].apply(tratar_texto)
  df = df[df['review_tratada']!='']
  df.drop_duplicates(subset=['review_tratada'],inplace=True)

  reviews_emb = modelo(df['review_tratada'].tolist())
  tema_emb = modelo([tema])

  similaridades = cosine_similarity(tema_emb,reviews_emb).flatten()
  top_indices = np.argsort(-similaridades)

  similar_reviews = df[['nota_review','review']].iloc[top_indices]

  similar_df = similar_reviews.head(200)

  nome_arquivo = f'reviews_similares_{tema}.csv'
  similar_reviews.to_csv(nome_arquivo)
  return similar_df, nome_arquivo

with gr.Blocks() as app:
  with gr.Row():
    gr.Markdown('## Encontrando as reviews mais similares ao tema')
  csv_entrada = gr.File(label='Envie o CSV com as reviews', file_types=['.csv'])
  tema_entrada = gr.Textbox(label='Digite o tema para busca (ex: "entrega")')

  botao = gr.Button('Clique para buscar as reviews')

  tabela_saida = gr.Dataframe(label='Top 200 reviews similares', headers=['Nota', 'Reviews'], interactive=False)
  arquivo_saida = gr.File(label='Baixar CSV ordenado com as reviews mais similares ao tema', interactive =False)

  botao.click(process_csv, inputs=[csv_entrada,tema_entrada], outputs=[tabela_saida,arquivo_saida])
app.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f92e24536848366782.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
